In [1]:
import pandas as pd
from tqdm import tqdm

import tensorflow as tf

/Users/uladzimir/app/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
%run "preprocessing.ipynb"

In [3]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

<h3>Model</h3>

In [21]:
x  = tf.placeholder(tf.float32, shape=[None, 128])
y_ = tf.placeholder(tf.float32, shape=[None, 2])

W_fc1 = weight_variable([128, 128])
b_fc1 = bias_variable([128])

W_fc2 = weight_variable([128, 2])
b_fc2 = bias_variable([2])

W_fc3 = weight_variable([256, 64])
b_fc3 = bias_variable([64])

W_fc4 = weight_variable([64, 2])
b_fc4 = bias_variable([2])

y_l1 = tf.matmul(x,W_fc1) + b_fc1
y_l2 = tf.matmul(y_l1,W_fc2) + b_fc2
# y_l3 = tf.matmul(y_l2,W_fc3) + b_fc3
# y_l4 = tf.matmul(y_l3,W_fc4) + b_fc4
y = y_l2

cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

train_step = tf.train.AdamOptimizer(0.2).minimize(cross_entropy)

correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [22]:
hangover['train'].reset()
hangover['test'].reset()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in tqdm(range(50)):
        batch = hangover['train'].next_batch(5)
        x_batch = batch['x']
        y_batch = np.array([[1,0] if i==0 else [0,1] for i in batch['y']]) 
        
        if i % 5 == 0:
            train_accuracy = accuracy.eval(feed_dict={
                x: x_batch, y_: y_batch})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        
        train_step.run(feed_dict={x: x_batch, y_: y_batch})

    y_batch_test = np.array([[1,0] if i==0 else [0,1] for i in hangover['test'].get_all('y')])
    print('test accuracy %g' % accuracy.eval(feed_dict={
        x: hangover['test'].get_all('x'), y_: y_batch_test}))

100%|██████████| 50/50 [00:00<00:00, 423.29it/s]

step 0, training accuracy 0.4
step 5, training accuracy 0.2
step 10, training accuracy 0.6
step 15, training accuracy 0.6
step 20, training accuracy 0.8
step 25, training accuracy 0.2
step 30, training accuracy 0.4
step 35, training accuracy 0.8
step 40, training accuracy 0.4
step 45, training accuracy 0.2
test accuracy 0.517241
